In [ ]:
from cedne import utils
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import copy
import os
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from scipy.stats import norm
from statsmodels.stats.multitest import multipletests

In [ ]:
if not os.path.isdir(utils.OUTPUT_DIR):
    os.makedirs(utils.OUTPUT_DIR)

In [ ]:
ntype = ['sensory', 'interneuron', 'motorneuron']
facecolors = ['#FF6F61', '#FFD700', '#4682B4']
ntype_color = dict(zip(ntype, facecolors))
ntype_pairs = [(nt1, nt2) for nt1 in ntype for nt2 in ntype]
colors= plt.cm.magma(np.linspace(0,1,len(ntype_pairs)))
type_color_dict = {p:color for (p,color) in zip(ntype_pairs, colors)}

In [ ]:
w = utils.makeWorm(chem_only=True)
nn = w.networks["Neutral"]

In [ ]:
triad_motifs = utils.return_triads()
motif = triad_motifs['030T']
motif = utils.nx.relabel_nodes(motif, {1:1, 2:3, 3:2})

mapping = (3,1)
chain_length = 3
hseq = utils.make_hypermotifs(motif, chain_length, [mapping])
hseq = utils.nx.convert_node_labels_to_integers(hseq, first_label=1, ordering='sorted', label_attribute='nodename')

In [ ]:
all_ffgs = nn.search_motifs(hseq)

In [ ]:
def return_motif_edgelabels(all_ffgs, hseq):    
    mot_edgelabels = {node:[] for node in hseq}
    for ffg in all_ffgs:
        nodelist = {node:None for node in hseq}
        for med, ned in ffg.items():
            for m,n in zip(med, ned):
                nodelist[m] = n.name
        for node in nodelist:
            mot_edgelabels[node].append(nodelist[node])

    count_edgelabels = {node:[] for node in hseq}
    for node in hseq:
        count_edgelabels[node] = Counter(mot_edgelabels[node])
    return mot_edgelabels, count_edgelabels

In [ ]:
mot_edgelabels, count_edgelabels = return_motif_edgelabels(all_ffgs, hseq)

In [ ]:
for e in sorted(mot_edgelabels):
    p = len(set(mot_edgelabels[e]))/len(nn.nodes)
    h = -p*np.log(p)
    print(h)

In [ ]:
ntype_color

In [ ]:
from cedne.utils import OUTPUT_DIR


f, ax = plt.subplots(nrows=len(count_edgelabels), figsize=(40,2*len(count_edgelabels)), layout='constrained', sharey=True)
for n, d in sorted(count_edgelabels.items(), key=lambda x:x[0]):
    name, height = zip(*[(name, height) for (name, height) in sorted(d.items(), key=lambda x:(ntype.index(nn.neurons[x[0]].type), x[0]))])
    print(sorted(d.items(), key=lambda x:x[1], reverse=True))
    print((d['AVAL'] + d['AVAR'] + d['AVBL'] + d['AVBR'] + d['AVDL'] + d['AVDR'] + d['AVEL'] + d['AVER'])/np.sum(height))
    color = [ntype_color[nn.neurons[neuname].type] for neuname in name]
    ax[n-1].bar(name, height/np.sum(height), color=color)
    ax[n-1].set_xticks(range(len(name)), name, rotation=45, fontsize='xx-large')
    for tick_label, c in zip(ax[n-1].get_xticklabels(), color):
        tick_label.set_color(c)
    ax[n-1].tick_params(axis='y', labelsize='xx-large')
    utils.simpleaxis(ax[n-1])
    ax[n-1].set_title(n, fontsize='xx-large')
f.supylabel("Fraction of matched neurons for node", fontsize='xx-large')
f.supxlabel("Neuron name", fontsize='xx-large')
plt.savefig(OUTPUT_DIR + "/HierarchicalSequence-neuronfractions.svg", transparent=True)
plt.show()

In [ ]:
## Junction convergence: #connections are the same, but the # neurons are different in each layer

In [ ]:
def convergence(layerCount):
    #return (1./(len(layerCount))) * np.sum([np.square((count/np.sum(list(layerCount.values())))) for count in layerCount.values()])
    #return np.mean([count/np.sum([v for v in layerCount.values()]) for count in layerCount.values()])

    return np.mean([count/np.sum([v for v in layerCount.values()]) for count in layerCount.values()])
    # H=0
    # # Hmax = np.log(302)
    # Hmax = 1
    # sum_conns = np.sum([v for v in layerCount.values()])
    # for count in layerCount.values():
    #     p = count/sum_conns
    #     H+= p*np.log(p)
    # return -H/Hmax

In [ ]:
real_convergence= np.array([convergence(count_edgelabels[k]) for k in range(1, len(hseq)+1)])
f, ax = plt.subplots(figsize=(1.5, 1.5))
ax.scatter(range(1, len(hseq)+1), real_convergence, color='k')
ax.set_xlabel("Layer")
ax.set_ylabel("Convergence (nats)")
ax.set_xticks(range(1, len(hseq)+1))
# ax.set_ylim((0,1))
utils.simpleaxis(ax)
plt.show()

In [ ]:
count_edgelabels[1]

In [ ]:
nrgraphs = 50
rand_counts = []
for g in range(nrgraphs):
    nnr = utils.randomize_graph(nn)
    rand_ffgs = nnr.search_motifs(hseq)
    rand_mot_edgelabels, rand_count_edgelabels = return_motif_edgelabels(rand_ffgs, hseq)
    rand_counts.append(rand_count_edgelabels)

In [ ]:
rand_convergence = [[convergence(rand_count_edgelabels[k]) for k in range(1, len(hseq)+1)] for rand_count_edgelabels in rand_counts]
rand_convergence = np.array(rand_convergence)

In [ ]:
zscore_sig_thres = 2.58
f, ax = plt.subplots(figsize=(1.5, 1.5))
conv_mu = np.mean(rand_convergence, axis=0)
conv_sigma = np.std(rand_convergence, axis=0)

ax.scatter(range(1, len(hseq)+1), real_convergence-conv_mu, color='purple')
# ax.errorbar(range(1, len(hseq)+1), conv_mu, yerr=conv_sigma, fmt= 'o', color='gray')

# zscore_sig = np.where(np.abs([(real_convergence[l]-conv_mu[l])/conv_sigma[l] for l in np.arange(0,len(hseq))])>zscore_sig_thres)[0]
# for x in zscore_sig:
#     ax.text(x=x+0.6, y=0.02, s="**", color='k')

ax.set_xlabel("Layer")
ax.set_ylabel("Convergence")
ax.set_xticks(range(1, len(hseq)+1))
ax.set_ylim((0,0.012))
utils.simpleaxis(ax)
plt.show()

In [ ]:
sum([v for v in rand_counts[4][2].values()])

In [ ]:
rand_counts[g][n]

In [ ]:
for n in range(1,8):
    for naam in name:
        rand_height = np.array([rand_counts[g][n][naam] for g in range(nrgraphs)])
        if naam not in rand_counts[0][n]:
            print(naam, rand_counts[0][n][naam])

In [ ]:
from cedne.utils import OUTPUT_DIR

# zscore_sig_thres =  1.96 ## 1.96 for 95% confidence, 2.58 for 99% confidence, 3.29 for 99.9% confidence, 4.9 for 99.999999% confidence
pval_thres = 0.05
zscores = {}
names = {}
rand_by_layer_n = {n:[] for n in range(1, len(hseq)+1)}
sig_neurs = []
f, ax = plt.subplots(nrows=len(count_edgelabels), figsize=(36,2*len(count_edgelabels)), layout='compressed', sharey=True)
for n, d in sorted(count_edgelabels.items(), key=lambda x:x[0]):
    zscores[n] = []
    name, height = zip(*[(name, height) for (name, height) in sorted(d.items(), key=lambda x:(ntype.index(nn.neurons[x[0]].type), x[0]))])
    # print(sorted(d.items(), key=lambda x:x[1], reverse=True))
    # print((d['AVAL'] + d['AVAR'] + d['AVBL'] + d['AVBR'] + d['AVDL'] + d['AVDR'] + d['AVEL'] + d['AVER'])/np.sum(height))
    color = [ntype_color[nn.neurons[neuname].type] for neuname in name]
    names[n] = name
    norm_height = height/np.sum(height)
    ax[n-1].bar(name, norm_height, color=color)
    print(len(name), len(norm_height))
    ax[n-1].set_xticks(range(len(name)), name, rotation=45, fontsize='xx-large', ha='center')
    for tick_label, c in zip(ax[n-1].get_xticklabels(), color):
        tick_label.set_color(c)
    ax[n-1].tick_params(axis='y', labelsize='xx-large')
    utils.simpleaxis(ax[n-1])
    ax[n-1].set_title(n, fontsize='xx-large')

    sum_heights = np.array([np.sum([v for v in rand_counts[g][n].values()]) for g in range(nrgraphs)])
    zscore_sig = []
    for i, naam in enumerate(name):
        rand_height = np.array([rand_counts[g][n][naam] for g in range(nrgraphs)])
        mu_rand = np.mean(rand_height/sum_heights)
        sig_rand = np.std(rand_height/sum_heights)

        # rand_by_layer_n[n].append(rand_height)
        ax[n-1].scatter(i, mu_rand, color='gray', alpha=0.3)
        ax[n-1].errorbar(i, mu_rand, yerr=sig_rand, color='k', alpha=0.3)#, s=5)

        zscores[n].append((norm_height[i] - mu_rand)/sig_rand)
        
        # if ((norm_height[i] - mu_rand)/sig_rand) > zscore_sig_thres:
        #     zscore_sig.append(i)
        #     sig_neurs.append(naam)
signames = []
for n in sorted(zscores):
    p_values_right = 1 - norm.cdf(zscores[n])
    # _, p_bh_right, _, _ = multipletests(p_values_right, method='bonferroni')
    _, p_bh_right, _, _ = multipletests(p_values_right, method='fdr_by')
    zscore_sig = np.where(p_bh_right<pval_thres)[0]
    
    for x in zscore_sig:
        ax[n-1].text(x=x, y=0.19, s="*", color='k', ha='center')
        signames.append(names[n][x])
    print(n,zscore_sig)
    ax[n-1].set_xlim((-1, len(zscores[n])+1))
sig_neurs = set(sig_neurs)
f.supylabel("Fraction of matched neurons for node", fontsize='xx-large')
f.supxlabel("Neuron name", fontsize='xx-large')
# plt.tight_layout()
plt.savefig(OUTPUT_DIR + "/HierarchicalSequence-neuronfractions-BY-correction-pval_0.05.svg", transparent=True)
plt.show()

In [ ]:
set(signames)

In [ ]:
zscore_sig_thres = 1.96 
f, ax = plt.subplots(figsize=(1.5,1.5))
ax.scatter(range(1, len(hseq)+1), real_convergence, color='purple')
conv_mu = np.mean(rand_convergence, axis=0)
conv_sigma = np.std(rand_convergence, axis=0)
ax.errorbar(range(1, len(hseq)+1), conv_mu, yerr=conv_sigma, fmt= 'o', color='gray')

zscore_sig = np.where(np.abs([(real_convergence[l]-conv_mu[l])/conv_sigma[l] for l in np.arange(0,len(hseq))])>zscore_sig_thres)[0]
for x in zscore_sig:
    ax.text(x=x+1, y=0.02, s="*", color='k', ha='center')

ax.set_xlabel("Layer")
ax.set_ylabel("Convergence")
ax.set_xticks(range(1, len(hseq)+1))
ax.set_yticks([0,0.01,0.02])
ax.set_ylim((0,0.02))
utils.simpleaxis(ax)
plt.savefig(OUTPUT_DIR + "/HierarchicalSequence-convergence.svg", transparent=True)
plt.show()

In [ ]:
zscore_sig_thres =  1.96 ## 1.96 for 95% confidence, 2.58 for 99% confidence, 3.29 for 99.9% confidence, 4.9 for 99.999999% confidence
rand_by_layer_n = {n:[] for n in range(1, len(hseq)+1)}
f, ax = plt.subplots(nrows=len(count_edgelabels), figsize=(36,2*len(count_edgelabels)), layout='compressed', sharey=True)
for n, d in sorted(count_edgelabels.items(), key=lambda x:x[0]):
    name, height = zip(*[(name, height) for (name, height) in sorted(d.items(), key=lambda x:(ntype.index(nn.neurons[x[0]].type), x[0]))])
    print(sorted(d.items(), key=lambda x:x[1], reverse=True))
    print((d['AVAL'] + d['AVAR'] + d['AVBL'] + d['AVBR'] + d['AVDL'] + d['AVDR'] + d['AVEL'] + d['AVER'])/np.sum(height))
    color = [ntype_color[nn.neurons[neuname].type] for neuname in name]
    norm_height = height/np.sum(height)
    ax[n-1].bar(name, norm_height, color=color)
    ax[n-1].set_xticks(range(len(name)), name, rotation=45, fontsize='xx-large', ha='center')
    for tick_label, c in zip(ax[n-1].get_xticklabels(), color):
        tick_label.set_color(c)
    ax[n-1].tick_params(axis='y', labelsize='xx-large')
    utils.simpleaxis(ax[n-1])
    ax[n-1].set_title(n, fontsize='xx-large')

    sum_heights = np.array([np.sum([v for v in rand_counts[g][n].values()]) for g in range(nrgraphs)])
    zscore_sig = []
    for i, naam in enumerate(name):
        rand_height = np.array([rand_counts[g][n][naam] for g in range(nrgraphs)])
        mu_rand = np.mean(rand_height/sum_heights)
        sig_rand = np.std(rand_height/sum_heights)

        # rand_by_layer_n[n].append(rand_height)
        ax[n-1].scatter(i, mu_rand, color='gray', alpha=0.3)
        ax[n-1].errorbar(i, mu_rand, yerr=sig_rand, color='k', alpha=0.3)#, s=5)
        if ((norm_height[i] - mu_rand)/sig_rand) > zscore_sig_thres:
            zscore_sig.append(i) 
    
    for x in zscore_sig:
        ax[n-1].text(x=x, y=0.19, s="*", color='k', ha='center')
    
    ax[n-1].set_xlim((-1, len(name)+1))

f.supylabel("Fraction of matched neurons for node", fontsize='xx-large')
f.supxlabel("Neuron name", fontsize='xx-large')
# plt.tight_layout()
plt.savefig(OUTPUT_DIR + "/HierarchicalSequence-neuronfractions.svg", transparent=True)
plt.show()

In [ ]:
f, ax = plt.subplots(nrows=len(count_edgelabels), figsize=(40,2*len(count_edgelabels)), layout='constrained', sharey=True)
for n, d in sorted(rand_counts[2].items(), key=lambda x:x[0]):
    name, height = zip(*[(name, height) for (name, height) in sorted(d.items(), key=lambda x:(ntype.index(nn.neurons[x[0]].type), x[0]))])
    print(sorted(d.items(), key=lambda x:x[1], reverse=True))
    print((d['AVAL'] + d['AVAR'] + d['AVBL'] + d['AVBR'] + d['AVDL'] + d['AVDR'] + d['AVEL'] + d['AVER'])/np.sum(height))
    color = [ntype_color[nn.neurons[neuname].type] for neuname in name]
    ax[n-1].bar(name, height/np.sum(height), color=color)
    ax[n-1].set_xticks(range(len(name)), name, rotation=45, fontsize='xx-large')
    for tick_label, c in zip(ax[n-1].get_xticklabels(), color):
        tick_label.set_color(c)
    ax[n-1].tick_params(axis='y', labelsize='xx-large')
    utils.simpleaxis(ax[n-1])
    ax[n-1].set_title(hseq.nodes[n]['nodename'], fontsize='xx-large')
f.supylabel("Fraction of matched neurons for node", fontsize='xx-large')
f.supxlabel("Neuron name", fontsize='xx-large')
# plt.savefig(OUTPUT_DIR + "/HierarchicalSequence-neuronfractions.svg", transparent=True)
plt.show()

In [ ]:
len(sig_neurs)

In [ ]:
utils.loadNeuropeptides(nn)

In [ ]:
conn_types_np = []
n = 'DVA'
for eid, conn in nn.neurons[n].get_connections(direction='both').items():
    if not conn.connection_type in ['gap-junction', 'chemical-synapse']:
        conn_types_np.append(conn.connection_type)
conn_types_np = set(conn_types_np)

In [ ]:
np_pre_post= {'pre':{}, 'post':{}}
for n in nn.neurons:
    for eid, conn in nn.neurons[n].get_connections(direction='both').items():
        if not conn.connection_type in ['gap-junction', 'chemical-synapse']:
            # if conn.connection_type in conn_types_np:
                if not conn.connection_type in np_pre_post['pre']:
                    np_pre_post['pre'][conn.connection_type] = []
                    np_pre_post['post'][conn.connection_type] = []
                np_pre_post['pre'][conn.connection_type].append(conn.pre.name)
                np_pre_post['post'][conn.connection_type].append(conn.post.name)

In [ ]:
for conn in conn_types_np:
    print(conn)
    print(len(Counter(np_pre_post['pre'][conn])), Counter(np_pre_post['pre'][conn]))
    print(len(Counter(np_pre_post['post'][conn])), Counter(np_pre_post['post'][conn]))

In [ ]:
conn1, conn2, conn3 = 'NLP_12_CKR_1', 'NLP_12_CKR_2', 'NTC_1_NTR_1'
print(len(set(np_pre_post['post'][conn1])), set(np_pre_post['post'][conn1]).intersection(set(np_pre_post['post'][conn3])))

In [ ]:
nlp-12; ckr-1, ckr-2

In [ ]:
by_np = {}
for eid, conn in nn.neurons['DVA'].get_connections(direction='both').items():
    if not conn.connection_type in ['gap-junction', 'chemical-synapse']:
        if not conn.connection_type in by_np:
            by_np[conn.connection_type] = []
        by_np[conn.connection_type]+= [conn.pre, conn.post]

In [ ]:
pre_n = {n:0 for n in nn.neurons}
post_n = {n:0 for n in nn.neurons}
for n in nn.neurons:
    for npep in np_pre_post['pre']:
        if n in np_pre_post['pre'][npep]:
            pre_n[n]+=1
        if n in np_pre_post['post'][npep]:
            post_n[n]+=1

In [ ]:
f, ax = plt.subplots(figsize=(2,2))
for n in nn.neurons:
    [post_n[n] if n not in sig_neurs]
        ax.hist(, color='gray')
    else:
        ax.hist(post_n, color='purple')
    # ax.scatter(pre_n[n], post_n[n], color='purple' if n in sig_neurs else 'gray')
utils.simpleaxis(ax)
plt.show()

In [ ]:
by_n = {"pre":[], "post":[]}
for n in nn.neurons:
    for cid, conn in nn.neurons[n].get_connections(direction='both').items():
        if not conn.connection_type in ['gap-junction', 'chemical-synapse']:
            conn.post
            print(conn.pre conn.connection_type)

In [ ]:
ha_by_n = {}
np_by_n = {}
num_conns = {}
for n in nn.neurons:
    num_conns[n] = 0
    ha_by_n[n] =[]
    np_by_n[n] = []
    conn_nps = {}
    for j, (conn_id, conn) in enumerate(nn.neurons[n].get_connections(direction='both').items()):
        if not conn.connection_type == 'chemical-synapse':
            if not conn.connection_type in conn_nps:
                conn_nps[conn.connection_type] = []
            conn_nps[conn.connection_type].append((conn_id[0], conn_id[1]))
            num_conns[n]+=1

    for neup, conns in conn_nps.items():
        ha_by_n[n].append(utils.hierarchical_alignment(conns))
        np_by_n[n].append(neup)

In [ ]:
f, ax = plt.subplots(figsize=(150,1), ncols=len(nn.neurons), sharey=True, layout='constrained')
for i, (n,v) in enumerate(sorted(num_conns.items(), key=lambda x:x[1])):
    ax[i].scatter(range(len(ha_by_n[n])), ha_by_n[n], color='k')
    utils.simpleaxis(ax[i])
    ax[i].set_xticks([])
    ax[i].set_title(n)
plt.show()

In [ ]:
for np, ha in zip(np_by_n['DVA'], ha_by_n['DVA']):
    print(np, ha)

In [ ]:
neu_interest = 'RMHL'
f, ax = plt.subplots(figsize=(1.5,1.5), layout='constrained')
xlabels = []
nodewise_out = {node:0 for node in mot_edgelabels}
nodewise_in = {node:0 for node in mot_edgelabels}

nodewise_out_chem = {node:0 for node in mot_edgelabels}
nodewise_in_chem = {node:0 for node in mot_edgelabels}

for j, (conn_id, conn) in enumerate(nn.neurons[neu_interest].get_connections(direction='out').items()):
    if not conn.connection_type == 'chemical-synapse':
        conn_nodes= [node for node in mot_edgelabels if conn_id[1].name in mot_edgelabels[node]]
        for node in mot_edgelabels:
            if node in conn_nodes:
                nodewise_out[node]+=1
    else:
        conn_nodes= [node for node in mot_edgelabels if conn_id[1].name in mot_edgelabels[node]]
        for node in mot_edgelabels:
            if node in conn_nodes:
                nodewise_out_chem[node]+=1

        # ax[0].scatter([j]*len(conn_nodes), conn_nodes)#,  conn.connection_type)
        # xlabels.append(conn_id[1].name)
# ax[0].set_xticks(range(len(xlabels)), xlabels)

xlabels = []
for j, (conn_id, conn) in enumerate(nn.neurons[neu_interest].get_connections(direction='in').items()):
    if not conn.connection_type == 'chemical-synapse':
        conn_nodes= [node for node in mot_edgelabels if conn_id[0].name in mot_edgelabels[node]]
        for node in mot_edgelabels:
            if node in conn_nodes:
                nodewise_in[node]+=1
    else:
        conn_nodes= [node for node in mot_edgelabels if conn_id[0].name in mot_edgelabels[node]]
        for node in mot_edgelabels:
            if node in conn_nodes:
                nodewise_in_chem[node]+=1
#         ax[1].scatter([j]*len(conn_nodes), conn_nodes)#,  conn.connection_type)
#         xlabels.append(conn_id[0].name)
# ax[1].set_xticks(range(len(xlabels)), xlabels)

ax.scatter(sorted(nodewise_out.keys()), [nodewise_out[node]/np.sum(list(nodewise_out.values())) for node in sorted(nodewise_out.keys())], label='out', color='orange')
ax.scatter(sorted(nodewise_in.keys()), [nodewise_in[node]/np.sum(list(nodewise_in.values())) for node in sorted(nodewise_in.keys())], label='in', color='purple')
utils.simpleaxis(ax)
ax.set_ylim((0,0.2))
ax.set_xticks(sorted(nodewise_out.keys()))
# f.legend(loc='outside upper right')
plt.show()
# conn_id[0].name, sorted([node for node in mot_edgelabels if conn_id[0].name in mot_edgelabels[node]])

In [ ]:
import scipy.stats as ss
f, ax = plt.subplots(figsize=(5,5), layout='constrained')
cm = plt.cm.viridis(np.linspace(0,1,len(hseq.nodes)))
colors = {n: cm[j] for (j, n) in enumerate(sorted(hseq.nodes))}
x,y = [], []
for node in nodewise_in:
    ax.scatter([nodewise_in[node]/np.sum(list(nodewise_in.values())) for node in sorted(nodewise_in.keys())], [nodewise_in_chem[node]/np.sum(list(nodewise_in_chem.values())) for node in sorted(nodewise_in_chem.keys())], marker='s', color=[colors[node] for node in sorted(nodewise_in_chem.keys())], label=sorted(nodewise_in_chem.keys()))
    ax.scatter([nodewise_out[node]/np.sum(list(nodewise_out.values())) for node in sorted(nodewise_out.keys())], [nodewise_out_chem[node]/np.sum(list(nodewise_out_chem.values())) for node in sorted(nodewise_out_chem.keys())], marker='o' , color=[colors[node] for node in sorted(nodewise_out_chem.keys())])
    x+= [nodewise_in[node]/np.sum(list(nodewise_in.values())) for node in sorted(nodewise_in.keys())]
    x+= [nodewise_out[node]/np.sum(list(nodewise_out.values())) for node in sorted(nodewise_out.keys())]
    y+=[nodewise_in_chem[node]/np.sum(list(nodewise_in_chem.values())) for node in sorted(nodewise_in_chem.keys())]
    y+=[nodewise_out_chem[node]/np.sum(list(nodewise_out_chem.values())) for node in sorted(nodewise_out_chem.keys())]
    utils.simpleaxis(ax)

result= ss.linregress(x,y)

xfit = np.linspace(0, 0.25, 100)
yfit = result.slope*xfit + result.intercept

# ax.plot([0,1], [0,1], 'k--')
ax.plot(xfit, yfit, 'r--')
ax.set_xlim((0.0,0.25))
ax.set_ylim((0.0,0.25))
ax.set_xticks([0, 0.1, 0.2])
ax.set_yticks([0, 0.1, 0.2])
f.supxlabel("Frac. Neuropeptide connections with node")
f.supylabel("Frac. Chemical synapses with node")
# plt.legend()
plt.show()

In [ ]:
result.slope, result.intercept, result.rvalue**2

In [ ]:
nodewise_in.values()

In [ ]:
all_seqha_neurons = list(set([neu for name_counts in count_edgelabels.values() for neu in name_counts.keys()]))
print(len(all_seqha_neurons))

In [ ]:
rsq_fit = {}
for n in all_seqha_neurons:
    nodewise_out = {node:0 for node in mot_edgelabels}
    nodewise_in = {node:0 for node in mot_edgelabels}

    nodewise_out_chem = {node:0 for node in mot_edgelabels}
    nodewise_in_chem = {node:0 for node in mot_edgelabels}

    for j, (conn_id, conn) in enumerate(nn.neurons[n].get_connections(direction='out').items()):
        if not conn.connection_type == 'chemical-synapse':
            conn_nodes= [node for node in mot_edgelabels if conn_id[1].name in mot_edgelabels[node]]
            for node in mot_edgelabels:
                if node in conn_nodes:
                    nodewise_out[node]+=1
        else:
            conn_nodes= [node for node in mot_edgelabels if conn_id[1].name in mot_edgelabels[node]]
            for node in mot_edgelabels:
                if node in conn_nodes:
                    nodewise_out_chem[node]+=1

            # ax[0].scatter([j]*len(conn_nodes), conn_nodes)#,  conn.connection_type)
            # xlabels.append(conn_id[1].name)
    # ax[0].set_xticks(range(len(xlabels)), xlabels)

    xlabels = []
    for j, (conn_id, conn) in enumerate(nn.neurons[n].get_connections(direction='in').items()):
        if not conn.connection_type == 'chemical-synapse':
            conn_nodes= [node for node in mot_edgelabels if conn_id[0].name in mot_edgelabels[node]]
            for node in mot_edgelabels:
                if node in conn_nodes:
                    nodewise_in[node]+=1
        else:
            conn_nodes= [node for node in mot_edgelabels if conn_id[0].name in mot_edgelabels[node]]
            for node in mot_edgelabels:
                if node in conn_nodes:
                    nodewise_in_chem[node]+=1

    x,y = [], []
    for node in nodewise_in:
        x+= [nodewise_in[node]/np.sum(list(nodewise_in.values())) for node in sorted(nodewise_in.keys())]
        x+= [nodewise_out[node]/np.sum(list(nodewise_out.values())) for node in sorted(nodewise_out.keys())]
        y+=[nodewise_in_chem[node]/np.sum(list(nodewise_in_chem.values())) for node in sorted(nodewise_in_chem.keys())]
        y+=[nodewise_out_chem[node]/np.sum(list(nodewise_out_chem.values())) for node in sorted(nodewise_out_chem.keys())]

    result= ss.linregress(x,y)
    rsq_fit[n] = result.rvalue**2


In [ ]:
rsq_fit_nanfilt = {k:v for k,v in rsq_fit.items() if not np.isnan(v) and k in sig_neurs}

In [ ]:
n, rsq = zip(*list(sorted(rsq_fit_nanfilt.items(), key=lambda x:x[1], reverse=True)))
f, ax = plt.subplots(figsize=(40,1.5))
ax.scatter(range(len(rsq)), rsq, color='purple')
ax.set_xlabel("Neuron")
ax.set_ylabel("R^2")
ax.set_xticks(range(len(n)), n, rotation=45, fontsize='xx-large')
ax.set_ylim((0,1))
utils.simpleaxis(ax)
plt.show()


In [ ]:
rsq_fit_nanfilt, ha_by_n_seqha = [], []
for k, v in rsq_fit.items():
    if not np.isnan(v):
        rsq_fit_nanfilt.append(v)
        ha_by_n_seqha.append(np.mean(np.abs(ha_by_n[k])))
        if k == 'DVA':
            print(k, np.mean(np.abs(ha_by_n[k])), v)
plt.scatter(ha_by_n_seqha, rsq_fit_nanfilt)
plt.show()

In [ ]:
common_conns_in = {}
common_conns_out = {}
for neu_interest in all_seqha_neurons:
    xlabels = []
    nodewise_out = {node:[] for node in mot_edgelabels}
    nodewise_in = {node:[] for node in mot_edgelabels}

    nodewise_out_chem = {node:[] for node in mot_edgelabels}
    nodewise_in_chem = {node:[] for node in mot_edgelabels}

    for j, (conn_id, conn) in enumerate(nn.neurons[neu_interest].get_connections(direction='out').items()):
        if not conn.connection_type == 'chemical-synapse':
            conn_nodes= [node for node in mot_edgelabels if conn_id[1].name in mot_edgelabels[node]]
            for node in mot_edgelabels:
                if node in conn_nodes:
                    nodewise_out[node].append(conn_id[1].name)
        else:
            conn_nodes= [node for node in mot_edgelabels if conn_id[1].name in mot_edgelabels[node]]
            for node in mot_edgelabels:
                if node in conn_nodes:
                    nodewise_out_chem[node].append(conn_id[1].name)

            # ax[0].scatter([j]*len(conn_nodes), conn_nodes)#,  conn.connection_type)
            # xlabels.append(conn_id[1].name)
    # ax[0].set_xticks(range(len(xlabels)), xlabels)

    xlabels = []
    for j, (conn_id, conn) in enumerate(nn.neurons[neu_interest].get_connections(direction='in').items()):
        if not conn.connection_type == 'chemical-synapse':
            conn_nodes= [node for node in mot_edgelabels if conn_id[0].name in mot_edgelabels[node]]
            for node in mot_edgelabels:
                if node in conn_nodes:
                    nodewise_in[node].append(conn_id[0].name)
        else:
            conn_nodes= [node for node in mot_edgelabels if conn_id[0].name in mot_edgelabels[node]]
            for node in mot_edgelabels:
                if node in conn_nodes:
                    nodewise_in_chem[node].append(conn_id[0].name)
    #         ax[1].scatter([j]*len(conn_nodes), conn_nodes)#,  conn.connection_type)
    #         xlabels.append(conn_id[0].name)
    # ax[1].set_xticks(range(len(xlabels)), xlabels)

    nodewise_in = {node: set(nodewise_in[node]) for node in nodewise_in}
    nodewise_out = {node: set(nodewise_out[node]) for node in nodewise_out}
    nodewise_in_chem = {node: set(nodewise_in_chem[node]) for node in nodewise_in_chem}
    nodewise_out_chem = {node: set(nodewise_out_chem[node]) for node in nodewise_out_chem}
    common_conns_in[neu_interest] = np.mean([len(nodewise_in[node].intersection(nodewise_in_chem[node]))/len(nodewise_in_chem[node]) for node in nodewise_in if len(nodewise_in_chem[node])>0])
    common_conns_out[neu_interest] = np.mean([len(nodewise_out[node].intersection(nodewise_out_chem[node]))/len(nodewise_out_chem[node]) for node in nodewise_out if len(nodewise_out_chem[node])>0])

In [ ]:
neus, vals = zip(* sorted([(k,v) for k,v in common_conns_out.items()if not np.isnan(v)], key=lambda x:x[1], reverse=True))
cols = ['purple' if n in ['DVA'] else 'gray' for n in neus]
f, ax = plt.subplots(figsize=(80,1.5))
ax.scatter(range(len(neus)), vals, color=cols)
ax.set_xlabel("Neuron")
ax.set_ylabel("Common connections")
ax.set_xticks(range(len(neus)), neus, rotation=45, fontsize='xx-large')
# ax.set_ylim((0,1))
utils.simpleaxis(ax)
plt.show()

In [ ]:
total_conns = {n:0 for n in all_seqha_neurons}
for n in all_seqha_neurons:
    for j, (conn_id, conn) in enumerate(nn.neurons[n].get_connections(direction='both').items()):
        if not conn.connection_type == 'chemical-synapse' or conn.connection_type == 'gap-junction':
            total_conns[n]+=1

In [ ]:
f, ax = plt.subplots(figsize=(80,1.5))
ax.scatter(range(len(total_conns)), [v for n,v in sorted(total_conns.items(), key=lambda x:x[1])], color='purple')
ax.set_xlabel("Neuron")
ax.set_ylabel("Total connections")
ax.set_xticks(range(len(total_conns)), [n for n,v in sorted(total_conns.items(), key=lambda x:x[1])], rotation=45, fontsize='xx-large')
plt.show()

In [ ]:
n, rsq = zip(*list(sorted(rsq_fit_nanfilt.items(), key=lambda x:x[1], reverse=True)))
f, ax = plt.subplots(figsize=(40,1.5))
ax.scatter(range(len(rsq)), rsq, color='purple')
ax.set_xlabel("Neuron")
ax.set_ylabel("R^2")
ax.set_xticks(range(len(n)), n, rotation=45, fontsize='xx-large')
ax.set_ylim((0,1))
utils.simpleaxis(ax)
plt.show()

In [ ]:

f, ax = plt.subplots(figsize=(1.5,1.5))
for node in nodewise_in:
    ax.scatter([(nodewise_in[node]+ nodewise_out[node])/(np.sum(list(nodewise_in.values())) + np.sum(list(nodewise_out.values()))) for node in sorted(nodewise_in.keys())],[(nodewise_in_chem[node]+ nodewise_out_chem[node])/(np.sum(list(nodewise_in_chem.values())) + np.sum(list(nodewise_out_chem.values()))) for node in sorted(nodewise_in_chem.keys())],  color='k')
    utils.simpleaxis(ax)
ax.set_xlim((0,0.2))
plt.show()

In [ ]:
npcount = {}
for n in nn.neurons:
    npcount[n] = 0
    for (conn_id, conn) in nn.neurons[n].get_connections().items():
        if not conn.connection_type == 'chemical-synapse':
            npcount[n] +=1#conn_id[0].name, conn_id[1].name, conn.connection_type)

In [ ]:
print(sorted(npcount.items(), key=lambda x:x[1], reverse=True))

In [ ]:
## Pooling
num_graphs = {}
hseq = utils.nx.convert_node_labels_to_integers(hseq, first_label=1, ordering='sorted', label_attribute='nodename')
utils.nx.draw(hseq, with_labels=True)
mappings = [(n,n) for n in sorted(hseq.nodes)]
max_chain_length = 6

for mapping in mappings:
    num_graphs[mapping] = []
    for l in range(2,max_chain_length+1):
        hm = utils.make_hypermotifs(hseq, l, [mapping])
        all_pooled_hseqs = nn.search_motifs(hm)
        num_graphs[mapping].append(len(all_pooled_hseqs))

In [ ]:
num_graphs

In [ ]:
colors = ['blue', 'gray']
f, ax = plt.subplots(figsize=(2.5,2.5), layout='constrained')
for m, mapping in enumerate(mappings):
    ax.scatter(np.arange(1,max_chain_length), num_graphs[mapping], color=colors[m], label=mapping)
    # ax.scatter(np.arange(1,len(num_graphs[mapping])+1), num_graphs[mapping], color='blue')
ax.set_xticks(np.arange(1,max_chain_length+1))
ax.set_yticks((0,5000,10000), ('0','5k','10k'))
# ax.yaxis.major.formatter._useMathText = True
ax.set_ylabel("# Networks")
ax.set_xlabel("# chained FFLs")
utils.simpleaxis(ax)
f.legend(loc='upper right', ncols=1, bbox_to_anchor=(1.1, 1))
# plt.savefig('FFL-chains-C_elegans.svg')
plt.show()

In [ ]:
mappings = [(1,1), (2,2), (3,3)]
num_graphs = {}
conn_types = {}
ntype_motif = {}
max_chain_length = 6
for mapping in mappings:
    num_graphs[mapping] = []
    ntype_motif[mapping] = {}
    conn_types[mapping] = {}
    for l in range(1,max_chain_length+1):
        ntype_motif[mapping][l] = {}
        conn_types[mapping][l] = {n:0 for n in ntype_pairs}
        hm = utils.make_hypermotifs(motif, l, [mapping])
        for m in hm.nodes:
            ntype_motif[mapping][l][m] = {n:0 for n in ntype}
        all_ffgs = nn.search_motifs(hm)
        num_graphs[mapping].append(len(all_ffgs))
        for sub in all_ffgs:
            for motif_edge, network_edge in sub.items():
                if all(n.type in ntype for n in network_edge):
                    for m,n in zip(motif_edge, network_edge):
                        ntype_motif[mapping][l][m][n.type] +=1
                    conn = (network_edge[0].type, network_edge[1].type)
                    conn_types[mapping][l][conn] +=1